In [1]:
val vars = "charge, charge_wo_dis, net_charge, net_charge_wo_dis"

val input = "prd2_dds.charge limit 1000"

var df = sqlContext.sql(s"select $vars from $input")

Waiting for a Spark session to start...

vars = charge, charge_wo_dis, net_charge, net_charge_wo_dis
input = prd2_dds.charge limit 1000
df = [charge: double, charge_wo_dis: double, net_charge: double, net_charge_wo_dis: double]


[charge: double, charge_wo_dis: double, net_charge: double, net_charge_wo_dis: double]

In [2]:
import org.apache.spark.sql.functions._

df = df.na.drop
        .withColumn("random_column", randn*3)
        .withColumn("label", expr("10 + 20*charge + 30*charge_wo_dis - 20*net_charge - net_charge_wo_dis + random_column"))
        .drop("random_column")

df = [charge: double, charge_wo_dis: double, net_charge: double, net_charge_wo_dis: double, label: double]


[charge: double, charge_wo_dis: double, net_charge: double, net_charge_wo_dis: double]

In [3]:
import org.apache.spark.ml.feature.VectorAssembler

var df = sqlContext.sql(s"select $vars from $input")

var columns = df.columns.toBuffer
columns -= "label"

val assembler = new VectorAssembler()
  .setInputCols(columns.toArray)
  .setOutputCol("features")

df = assembler.transform(df)
columns.foreach(col => {df = df.drop(col)})

val Array(train, test) = df.randomSplit(Array(0.7, 0.3), seed = 123)

columns = ArrayBuffer(charge, charge_wo_dis, net_charge, net_charge_wo_dis)
assembler = vecAssembler_09b2d2d9c5d6
df = [label: double, features: vector]
train = [label: double, features: vector]
test = [label: double, features: vector]


[label: double, features: vector]

In [4]:
import org.apache.spark.ml.regression.LinearRegression

val lr = new LinearRegression()
  .setRegParam(0.3)

val lrModel = lr.fit(train)

lr = linReg_f0d34f93bea4
lrModel = linReg_f0d34f93bea4


linReg_f0d34f93bea4

In [5]:
println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

val trainingSummary = lrModel.summary
println(s"numIterations: ${trainingSummary.totalIterations}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"r2: ${trainingSummary.r2}")

trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@4670153b


Coefficients: [3.2452805441963943,12.854794608981743,3.8276264837046803,15.168657983361008] Intercept: 11.054638359342487
numIterations: 1
RMSE: 3.076037095196053
r2: 0.9999853840607333


In [6]:
val results = lrModel.transform(test)

results.show(false)

results: org.apache.spark.sql.DataFrame = [label: double, features: vector, prediction: double]


+------------------+-----------------------+------------------+
|label             |features               |prediction        |
+------------------+-----------------------+------------------+
|17.044277454247627|[0.3,0.3,0.25,0.254237]|20.69800162593785 |
|17.92339429266562 |[0.5,0.5,0.42,0.423729]|27.139679337719098|
|18.158274970339093|[0.5,0.5,0.42,0.423729]|27.139679337719098|
|19.439520829081825|[0.5,0.5,0.42,0.423729]|27.139679337719098|
|19.64731062764445 |[0.3,0.3,0.25,0.254237]|20.69800162593785 |
|20.080913987153533|[0.5,0.5,0.42,0.423729]|27.139679337719098|
|20.1386868316536  |[0.5,0.5,0.42,0.423729]|27.139679337719098|
|20.13900809225025 |[0.5,0.5,0.42,0.423729]|27.139679337719098|
|20.89338948807242 |[0.5,0.5,0.42,0.423729]|27.139679337719098|
|20.935095696986764|[0.5,0.5,0.42,0.423729]|27.139679337719098|
|20.9688438676972  |[0.5,0.5,0.42,0.423729]|27.139679337719098|
|20.99205647305491 |[0.5,0.5,0.42,0.423729]|27.139679337719098|
|21.070159567714896|[0.5,0.5,0.42,0.4237

In [7]:
lrModel.save("lrModel")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/cloudera/parcels/CDH-5.10.2-1.cdh5.10.2.p0.5/jars/hive-exec-1.1.0-cdh5.10.2.jar!/shaded/parquet/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/cloudera/parcels/CDH-5.10.2-1.cdh5.10.2.p0.5/jars/hive-jdbc-1.1.0-cdh5.10.2-standalone.jar!/shaded/parquet/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/cloudera/parcels/CDH-5.10.2-1.cdh5.10.2.p0.5/jars/parquet-format-2.1.0-cdh5.10.2.jar!/shaded/parquet/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/cloudera/parcels/CDH-5.10.2-1.cdh5.10.2.p0.5/jars/parquet-hadoop-bundle-1.5.0-cdh5.10.2.jar!/shaded/parquet/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/cloudera/parcels/CDH-5.10.2-1.cdh5.10.2.p0.5/jars/parquet-pig-bundle-1.5.0-cdh5.10.2.jar!/shaded/parquet/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org

In [8]:
import org.apache.spark.ml.regression.LinearRegressionModel

val loadedModel = LinearRegressionModel.load("lrModel")

loadedModel = linReg_080d21081642


linReg_080d21081642

In [9]:
val resultsFromLoaded = loadedModel.transform(test)

resultsFromLoaded.show(false)

resultsFromLoaded: org.apache.spark.sql.DataFrame = [charge: double, charge_wo_dis: double, net_charge: double, net_charge_wo_dis: double, label: double, features: vector, prediction: double]


+------+-------------+----------+-----------------+------------------+----------------------+------------------+
|charge|charge_wo_dis|net_charge|net_charge_wo_dis|label             |features              |prediction        |
+------+-------------+----------+-----------------+------------------+----------------------+------------------+
|0.0   |0.3          |0.0       |0.254237         |12.974408468059341|[0.0,0.3,0.0,0.254237]|18.721849221198553|
|0.0   |0.3          |0.0       |0.254237         |14.427797340419037|[0.0,0.3,0.0,0.254237]|18.721849221198553|
|0.0   |0.3          |0.0       |0.254237         |14.455813284180346|[0.0,0.3,0.0,0.254237]|18.721849221198553|
|0.0   |0.3          |0.0       |0.254237         |19.346970712887245|[0.0,0.3,0.0,0.254237]|18.721849221198553|
|0.0   |0.3          |0.0       |0.254237         |22.040643086808526|[0.0,0.3,0.0,0.254237]|18.721849221198553|
|0.0   |1.0          |0.0       |0.847458         |39.07093939954831 |[0.0,1.0,0.0,0.847458]|39.